In [59]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key
%matplotlib notebook

# 1. Load Data

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/WeatherPy_challenge.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Weather description,Rain inches(last 3 hrs),Snow inches(last 3 hrs)
0,0,Champasak,14.75,106.00,70.92,39,0,3.04,LA,2020-02-23 15:38:15,clear sky,0.0,0.0
1,1,Tessalit,20.20,1.01,92.82,7,0,9.04,ML,2020-02-23 15:38:15,clear sky,0.0,0.0
2,2,Saint-Philippe,-21.36,55.77,80.60,69,75,4.70,RE,2020-02-23 15:38:15,broken clouds,0.0,0.0
3,3,Ushuaia,-54.80,-68.30,50.00,76,75,4.76,AR,2020-02-23 15:36:57,drizzle,0.0,0.0
4,4,Luderitz,-26.65,15.16,66.22,79,0,22.62,NaN,2020-02-23 15:38:16,clear sky,0.0,0.0


In [3]:
city_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550 entries, 0 to 549
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   City_ID                  550 non-null    int64  
 1   City                     550 non-null    object 
 2   Lat                      550 non-null    float64
 3   Lng                      550 non-null    float64
 4   Max Temp                 550 non-null    float64
 5   Humidity                 550 non-null    int64  
 6   Cloudiness               550 non-null    int64  
 7   Wind Speed               550 non-null    float64
 8   Country                  547 non-null    object 
 9   Date                     550 non-null    object 
 10  Weather description      550 non-null    object 
 11  Rain inches(last 3 hrs)  550 non-null    float64
 12  Snow inches(last 3 hrs)  550 non-null    float64
dtypes: float64(6), int64(3), object(4)
memory usage: 56.0+ KB


# 2.  Develop Dataframe Based on Criteria

In [4]:
#Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
# Ask the customer to add a rain and snow values
Rain= input("Do you want it to be raining? (yes/no)?")
Snow= input("Do you want it to be snowing? (yes/no)?")

What is the minimum temperature you would like for your trip? 25
What is the maximum temperature you would like for your trip? 75
Do you want it to be raining? (yes/no)?yes
Do you want it to be snowing? (yes/no)?yes


In [5]:
# filter the dataset to find the cities
preferred_cities_df=city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]

if Rain=="no" and Snow=="no":
    preferred_cities_df=preferred_cities_df.loc[(preferred_cities_df["Rain inches(last 3 hrs)"]<=0) & (preferred_cities_df["Snow inches(last 3 hrs)"]<=0)]                                                                          

elif Rain=="no":
    preferred_cities_df=preferred_cities_df.loc[(preferred_cities_df["Rain inches(last 3 hrs)"]<=0),:]
                                                 
elif Snow=="no":
    preferred_cities_df=preferred_cities_df.loc[(preferred_cities_df["Snow inches(last 3 hrs)"]<=0),:]                                             

else:
    preferred_cities_df

preferred_cities_df.info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 264 entries, 0 to 547
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   City_ID                  264 non-null    int64  
 1   City                     264 non-null    object 
 2   Lat                      264 non-null    float64
 3   Lng                      264 non-null    float64
 4   Max Temp                 264 non-null    float64
 5   Humidity                 264 non-null    int64  
 6   Cloudiness               264 non-null    int64  
 7   Wind Speed               264 non-null    float64
 8   Country                  263 non-null    object 
 9   Date                     264 non-null    object 
 10  Weather description      264 non-null    object 
 11  Rain inches(last 3 hrs)  264 non-null    float64
 12  Snow inches(last 3 hrs)  264 non-null    float64
dtypes: float64(6), int64(3), object(4)
memory usage: 28.9+ KB


In [6]:
# drop null rows
preferred_cities_df=preferred_cities_df.dropna()

In [7]:
preferred_cities_df.count()

City_ID                    263
City                       263
Lat                        263
Lng                        263
Max Temp                   263
Humidity                   263
Cloudiness                 263
Wind Speed                 263
Country                    263
Date                       263
Weather description        263
Rain inches(last 3 hrs)    263
Snow inches(last 3 hrs)    263
dtype: int64

# 3.  Get Travel Data Information

In [8]:
# Print results rain and snow count
rain_count=preferred_cities_df.loc[preferred_cities_df["Rain inches(last 3 hrs)"]>0, ["Rain inches(last 3 hrs)"]].count()
snow_count=preferred_cities_df.loc[preferred_cities_df["Snow inches(last 3 hrs)"]>0, ["Snow inches(last 3 hrs)"]].count()
print(f'rain count:{rain_count},snow count:{snow_count}')

rain count:Rain inches(last 3 hrs)    16
dtype: int64,snow count:Snow inches(last 3 hrs)    9
dtype: int64


In [9]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp","Weather description", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Max Temp,Weather description,Lat,Lng,Hotel Name
0,Champasak,LA,70.92,clear sky,14.75,106.00,
3,Ushuaia,AR,50.00,drizzle,-54.80,-68.30,
6,Bethel,US,48.99,clear sky,41.37,-73.41,
8,Kapaa,US,66.20,clear sky,22.08,-159.32,
9,Teguise,ES,73.40,haze,29.06,-13.56,


In [10]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [11]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [47]:
hotel_df.head()

,City,Country,Max Temp,Weather description,Lat,Lng,Hotel Name
0,Champasak,LA,70.92,clear sky,14.75,106.00,
3,Ushuaia,AR,50.00,drizzle,-54.80,-68.30,Albatros Hotel
6,Bethel,US,48.99,clear sky,41.37,-73.41,Courtyard by Marriott Danbury
8,Kapaa,US,66.20,clear sky,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
9,Teguise,ES,73.40,haze,29.06,-13.56,B&B La Casona de Nazaret


In [48]:
hotel_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 263 entries, 0 to 547
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City                 263 non-null    object 
 1   Country              263 non-null    object 
 2   Max Temp             263 non-null    float64
 3   Weather description  263 non-null    object 
 4   Lat                  263 non-null    float64
 5   Lng                  263 non-null    float64
 6   Hotel Name           263 non-null    object 
dtypes: float64(3), object(4)
memory usage: 26.4+ KB


In [54]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current weather description</dt><dd>{Weather description}</dd>
</dl>
"""

In [55]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [61]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=100,point_radius=1)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [62]:
# Create the output file (CSV).
output_data_file = "weather_data/WeatherPy_vacation.csv"
# Export the City_Data into a CSV.
hotel_df.to_csv(output_data_file, index_label="City_ID")

End of File